# 🧪 Hands-On Lab for Section 2: Foundations of RAG Architecture

**Lab Title:**
Implementing the Retrieval and Generation Pipeline with FAISS and OpenAI LLM

## Objective:
By the end of this lab, learners will:
- Build a retrieval subsystem that finds relevant text from a vector database.
- Build a generation subsystem that uses LLMs to create context-aware answers.
- Combine both to form a working RAG mini-architecture.

## Install dependencies

In [1]:
!pip install openai faiss-cpu tiktoken pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## Import libraries

In [20]:
import numpy as np
import faiss
from openai import OpenAI

# Set your OpenRouter API key here
OPENROUTER_API_KEY = "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"

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

## 🧩 Part 1 – The Retrieval Process

**Goal:** Understand how embeddings are created, stored, and searched using semantic similarity.

### Create a mini-dataset

In [21]:
knowledge_base = [
    "RAG stands for Retrieval-Augmented Generation.",
    "In RAG, relevant context is fetched before generating answers.",
    "Vector databases such as FAISS store embeddings for fast search.",
    "Embeddings represent text meaning as numerical vectors.",
    "Combining retrieval and generation improves accuracy and factuality."
]

print("Knowledge base created with", len(knowledge_base), "documents")

Knowledge base created with 5 documents


### Generate embeddings and store in FAISS

In [22]:
# Helper function to get embeddings using OpenRouter
def get_embedding(text, model="text-embedding-3-small"):
    """Generate embeddings using OpenAI's embedding model via OpenRouter"""
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

# Generate embeddings for all documents in knowledge base
print("Generating embeddings...")
vectors = [get_embedding(text) for text in knowledge_base]

# Create FAISS index
dimension = len(vectors[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(vectors).astype("float32"))

print(f"✅ FAISS index created with {index.ntotal} vectors of dimension {dimension}")

Generating embeddings...
✅ FAISS index created with 5 vectors of dimension 1536


### Test retrieval

In [27]:
query = "How does RAG improve AI accuracy?"

# Get embedding for the query
query_vector = get_embedding(query)

# Search for top 2 most similar documents
D, I = index.search(np.array([query_vector]).astype("float32"), k=2)

# Retrieve the matching documents
retrieved_docs = [knowledge_base[i] for i in I[0]]

print("Top matches:\n")
for idx, doc in enumerate(retrieved_docs, 1):
    print(f"{idx}. {doc}")

Top matches:

1. RAG stands for Retrieval-Augmented Generation.
2. In RAG, relevant context is fetched before generating answers.


## 🧠 Part 2 – The Generation Process

**Goal:** Use an LLM to generate context-aware answers based on retrieved data.

### Combine retrieved context and query

In [28]:
context = " ".join(retrieved_docs)
prompt = f"Use the context below to answer precisely.\n\nContext:\n{context}\n\nQuestion: {query}"

print("Prompt sent to LLM:")
print("-" * 60)
print(prompt)
print("-" * 60)

Prompt sent to LLM:
------------------------------------------------------------
Use the context below to answer precisely.

Context:
RAG stands for Retrieval-Augmented Generation. In RAG, relevant context is fetched before generating answers.

Question: How does RAG improve AI accuracy?
------------------------------------------------------------


### Generate the answer

In [30]:
response = client.chat.completions.create(
    model="qwen/qwen-2-7b-instruct:free",  # Valid free model on OpenRouter
    messages=[{"role": "user", "content": prompt}]
)

print("Generated Answer:\n")
print(response.choices[0].message.content)

NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found for qwen/qwen-2-7b-instruct:free.', 'code': 404}, 'user_id': 'user_2wfJyNuF5yzdHhNuld6ltJBaBZP'}

## ⚙️ Part 3 – Putting It Together

**Goal:** Build a complete RAG function that takes a user question and returns a contextual answer.

### Define a complete RAG pipeline function

In [ ]:
def rag_pipeline(question, k=2):
    """
    Complete RAG pipeline: Retrieval + Generation
    
    Args:
        question: User's question
        k: Number of top documents to retrieve
    
    Returns:
        Generated answer based on retrieved context
    """
    # Step 1: Retrieve relevant documents
    q_vec = get_embedding(question)
    D, I = index.search(np.array([q_vec]).astype("float32"), k=k)
    context = " ".join([knowledge_base[i] for i in I[0]])
    
    # Step 2: Create prompt with context
    prompt = f"Context:\n{context}\n\nQuestion: {question}"
    
    # Step 3: Generate answer
    result = client.chat.completions.create(
        model="qwen/qwen-2-7b-instruct:free",  # Free model on OpenRouter
        messages=[{"role": "user", "content": prompt}]
    )
    
    return result.choices[0].message.content

print("✅ RAG pipeline function ready!")

✅ RAG pipeline function ready!


### Ask your AI

In [19]:
print("Question 1: What are vector databases used for?")
print("-" * 60)
answer1 = rag_pipeline("What are vector databases used for?")
print(answer1)
print("\n" + "=" * 60 + "\n")

print("Question 2: Explain the main advantage of RAG.")
print("-" * 60)
answer2 = rag_pipeline("Explain the main advantage of RAG.")
print(answer2)

Question 1: What are vector databases used for?
------------------------------------------------------------


NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found for meta-llama/llama-3.1-8b-instruct:free.', 'code': 404}, 'user_id': 'user_2wfJyNuF5yzdHhNuld6ltJBaBZP'}

## 🧭 Bonus Challenge

Try these experiments:

1. **Replace the toy dataset** with your own text (PDF excerpts, company docs, or Wikipedia articles).
2. **Experiment with different k values** (top-k retrieval) to see how it affects the quality of answers.
3. **Compare responses with and without retrieval** to see the impact on factual accuracy.

### Example: Compare with and without RAG

In [ ]:
test_question = "What does RAG stand for?"

# Without RAG (direct LLM query)
print("WITHOUT RAG:")
print("-" * 60)
response_no_rag = client.chat.completions.create(
    model="qwen/qwen-2-7b-instruct:free",  # Free model on OpenRouter
    messages=[{"role": "user", "content": test_question}]
)
print(response_no_rag.choices[0].message.content)

print("\n" + "=" * 60 + "\n")

# With RAG
print("WITH RAG:")
print("-" * 60)
response_with_rag = rag_pipeline(test_question)
print(response_with_rag)

print("\n" + "=" * 60)
print("\n✅ Notice how RAG provides more specific, context-grounded answers!")

## 🎯 Summary

You've successfully built a complete RAG system! Here's what you accomplished:

1. ✅ **Retrieval**: Created embeddings and stored them in FAISS for semantic search
2. ✅ **Generation**: Used an LLM to generate context-aware answers
3. ✅ **Integration**: Combined both into a single `rag_pipeline()` function
4. ✅ **Comparison**: Saw the difference between RAG and non-RAG responses

This is your first fully functional RAG mini-system!